In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from tqdm import tqdm
import ast

import random

from multiprocessing import Pool

In [5]:
secuencias_entrenamiento_df = pd.read_csv('../../Data/secuencias_entrenamiento.csv')
secuencias_entrenamiento_df

,session_id,item_id_salida,secuencia_items_ids,secuencia_fechas
0,3,15085,"[9655, 9655]","['2020-12-18 21:19:48.093', '2020-12-18 21:25:..."
1,13,18626,[15654],['2020-03-13 19:35:27.136']
2,18,24911,"[4026, 2507, 18316]","['2020-08-26 19:15:47.232', '2020-08-26 19:16:..."
3,19,12534,"[19896, 27937, 12804, 25772, 6341, 23687, 8316...","['2020-11-02 16:30:36.378', '2020-11-02 16:30:..."
4,24,13226,"[2927, 2927, 16064, 11662, 434, 18539, 10414, ...","['2020-02-26 17:22:48.903', '2020-02-26 17:23:..."
...,...,...,...,...
999995,4439986,2915,"[23502, 13914, 13403, 19310, 12373, 27733]","['2021-05-13 11:46:56.606', '2021-05-13 11:48:..."
999996,4439990,8786,"[22093, 17429, 1389, 1131, 4062, 26011, 4028, ...","['2020-08-22 11:38:48.785', '2020-08-22 11:39:..."
999997,4439994,21630,[25357],['2020-11-27 20:08:37.262']
999998,4439999,16962,"[6007, 8502, 13056, 2173, 8433, 1299, 15853]","['2020-11-27 10:52:12.577', '2020-11-27 10:53:..."


In [8]:
items_onehot_encoded_df = pd.read_csv('../../Data/items_onehot_encoded.csv')
items_onehot_encoded_df

,item_id,1_60,1_143,1_358,1_461,1_517,1_771,2_53,2_56,2_83,...,72_665,72_671,72_712,72_747,72_748,72_751,72_829,72_883,73_91,73_544
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23687,28140,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
23688,28141,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23689,28142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [41]:
def crear_pares_positivos_con_contexto_global(secuencias_df):
    
    columna_1 = []
    columna_2 = []
    
    for index, row in tqdm(secuencias_df.iterrows()):
        
        secuencia_items = ast.literal_eval(row.secuencia_items_ids)
        
        item_final = row.item_id_salida
        
        # Creacion de los pares positivos a partir de la secuencia de vistas
        
        if len(secuencia_items) > 1:
            
            columna_1.extend(secuencia_items[:-1])
            columna_2.extend(secuencia_items[1:])

        # Anadir el contexto global a los pares positivos

        columna_1.extend(secuencia_items)
        columna_2.extend([item_final]*len(secuencia_items))
        
        
    
    pares_positivos_df = pd.DataFrame(list(zip(columna_1,columna_2)), columns = ['Id_1','Id_2'])


    return pares_positivos_df


In [42]:
pares_positivos = crear_pares_positivos_con_contexto_global(secuencias_entrenamiento_df)
pares_positivos

1000000it [00:43, 23227.39it/s]


,Id_1,Id_2
0,9655,9655
1,9655,15085
2,9655,15085
3,15654,18626
4,4026,2507
...,...,...
8487635,14303,16631
8487636,19539,16631
8487637,20409,16631
8487638,27852,16631


In [44]:
pares_positivos = pares_positivos.drop_duplicates()
pares_positivos

,Id_1,Id_2
0,9655,9655
1,9655,15085
3,15654,18626
4,4026,2507
5,2507,18316
...,...,...
8487634,26067,16631
8487635,14303,16631
8487636,19539,16631
8487637,20409,16631


In [48]:
pares_positivos = pares_positivos[pares_positivos['Id_1'] != pares_positivos['Id_2']]
pares_positivos

,Id_1,Id_2
1,9655,15085
3,15654,18626
4,4026,2507
5,2507,18316
6,4026,24911
...,...,...
8487634,26067,16631
8487635,14303,16631
8487636,19539,16631
8487637,20409,16631


In [50]:
pares_positivos.to_csv('../../Data/pares_positivos_ids.csv', index=False)

In [53]:
items_ids = list(items_onehot_encoded_df.item_id)
len(items_ids)

23691

In [59]:
def crear_pares_negativos_aleatorios(item_ids, cantidad_de_pares = 100):
    
    columna_1 = []
    columna_2 = []
    
    for i in range(cantidad_de_pares):
        
        columna_1.append(random.choice(item_ids))
        columna_2.append(random.choice(item_ids))
        
    pares_negativos_df = pd.DataFrame(list(zip(columna_1,columna_2)), columns = ['Id_1','Id_2'])

    return pares_negativos_df

In [66]:
cantidad_pares_negativos = 4600000

pares_negativos = crear_pares_negativos_aleatorios(items_ids, cantidad_pares_negativos)
pares_negativos = pares_negativos.drop_duplicates()
pares_negativos = pares_negativos[pares_negativos['Id_1'] != pares_negativos['Id_2']]
pares_negativos = pares_negativos.iloc[:4513628]
pares_negativos

,Id_1,Id_2
0,2379,9050
1,10138,9537
2,12381,1716
3,5126,7195
4,16240,23369
...,...,...
4532077,24662,9427
4532078,11572,18734
4532079,9809,20969
4532080,25278,27264


In [68]:
pares_negativos.to_csv('../../Data/pares_negativos_ids.csv', index=False)